In [21]:
from astropy.io import ascii, fits
from astropy.table import Table
import pylab as plt
%matplotlib inline
from astropy import wcs
import seaborn as sns
import glob
import arviz as az

import numpy as np
import xidplus
from xidplus import moc_routines
import pickle
import os
from ESB_functions import *
sns.set_context("paper")
from astropy.coordinates import SkyCoord
from astropy import units as u
import os
import jax.numpy as jnp
import pyvo as vo


In [3]:
field=['Lockman-SWIRE']
path_to_SPIRE=['/Volumes/pdh_storage/dmu_products/dmu19/dmu19_HELP-SPIRE-maps/data/']
#path_to_SPIRE=['/mnt/lustre/users/astro/pdh21/Extreme_submm_SB_gals/data/']
redshift_file=["/Volumes/pdh_storage/dmu_products/dmu24/dmu24_Lockman-SWIRE/data/master_catalogue_Lockman-SWIRE_20170710_photoz_20170802_r_and_irac1_optimised_UPDATED_IDs_20180219.fits"]
#redshift_file=["/mnt/lustre/users/astro/pdh21/Extreme_submm_SB_gals/data/master_catalogue_Lockman-SWIRE_20170710_photoz_20170802_r_and_irac1_optimised_UPDATED_IDs_20180219.fits"]

In [14]:
esb=Table.read('../../../data/MRR2018_tables/{}_web.csv'.format(field[0]),format='ascii',encoding='utf-8')

In [15]:
esb['redshift']=np.zeros((len(esb)))
for b in ['250','350','500']:
    esb['f_spire_{}'.format(b)]=np.zeros((len(esb)))
    esb['ferr_spire_{}'.format(b)]=np.zeros((len(esb)))


In [16]:
for i in range(1,len(esb)):
    source=[i]
    model = 'model2'

    try:
        data=az.InferenceData.from_netcdf('./output/{}/posterior_{}_{}_{}.nc'.format(field[0],field[0],source[0],model))
        prior_list=Table.read('./output/{}/prior_list_{}_{}_{}.ascii'.format(field[0],field[0],source[0],model),format='ascii',encoding='utf-8')
        priors,_ = xidplus.load('./output/{}/prior_{}_{}_{}.pkl'.format(field[0],field[0],source[0],model))
    except FileNotFoundError:
        try:
            model = 'model1'
            data=az.InferenceData.from_netcdf('./output/{}/posterior_{}_{}_{}.nc'.format(field[0],field[0],source[0],model))
            prior_list=Table.read('./output/{}/prior_list_{}_{}_{}.ascii'.format(field[0],field[0],source[0],model),format='ascii',encoding='utf-8')
            priors,_ = xidplus.load('./output/{}/prior_{}_{}_{}.pkl'.format(field[0],field[0],source[0],model))
        except FileNotFoundError:
            print('No fit')
            continue
        
    c = SkyCoord(ra=[esb['RA'][source[0]]]*u.degree, dec=[esb['Dec'][source[0]]]*u.degree)
    #find the sources that are within a certain separation of the extreme starburst candidate
    sep=12
    separation=c.separation(SkyCoord(priors[0].sra,priors[0].sdec)).arcsec
    contrib_sources=separation<sep
    ind = prior_list['help_id']==priors[0].ID[np.argmin(separation)]
    emulator_path=['/Users/pdh21/Google_Drive/WORK/XID_plus/docs/notebooks/examples/SED_emulator/CIGALE_emulator_20210420_log10sfr_uniformAGN_z.npz']


    def create_phys_prior(priors,prior_list,emulator_path):
        z_mu=np.empty((priors[1].nsrc))
        z_sig=np.empty((priors[1].nsrc))

        for x,i in enumerate(priors[1].ID):

            ind=prior_list['help_id']==str(i)
            z_mu[x]=prior_list['redshift'][ind]
            z_sig[x]=prior_list['redshift_unc'][ind]



        sfr_mu=1.5*z_mu+1.0
        sfr_sig=np.full(priors[1].nsrc,0.5)

        agn_frac_mu=np.full(priors[1].nsrc,-1)
        agn_frac_sig=np.full(priors[1].nsrc,1)
        params_mu=np.vstack((sfr_mu,z_mu)).T
        params_sig=np.vstack((sfr_sig,z_sig)).T
        params_names=['log10sfr','z']

        return xidplus.hier_prior(priors[1].ID,params_mu,params_sig,params_names,emulator_path[0])

    hier_params={'m_mu':0.8,'m_sig':0.4,'c_mu':-1.0,'c_sig':0.5,'sfr_disp':0.5}
    phys_prior=create_phys_prior(priors,prior_list,emulator_path)


    #stack params and make vector ready to be used by emualator
    params = jnp.stack((data.posterior.sfr.values.reshape(data.posterior.chain.size * data.posterior.draw.size,data.posterior.src.size).T, data.posterior.agn.values.reshape(data.posterior.chain.size * data.posterior.draw.size,data.posterior.src.size).T,data.posterior.redshift.values.reshape(data.posterior.chain.size * data.posterior.draw.size,data.posterior.src.size).T)).T
    # Use emulator to get fluxes. As emulator provides log flux, convert.
    src_f = np.array(jnp.exp(phys_prior.emulator['net_apply'](phys_prior.emulator['params'], params)))


    #stack params and make vector ready to be used by emualator
    params_prior = jnp.stack((data.prior.sfr.values.reshape(data.prior.chain.size * data.prior.draw.size,data.prior.src.size).T, data.prior.agn.values.reshape(data.prior.chain.size * data.prior.draw.size,data.prior.src.size).T,data.prior.redshift.values.reshape(data.prior.chain.size * data.prior.draw.size,data.prior.src.size).T)).T

    # Use emulator to get fluxes. As emulator provides log flux, convert.
    src_f_prior = np.array(jnp.exp(phys_prior.emulator['net_apply'](phys_prior.emulator['params'], params_prior)))
    esb['redshift'][i]=data.posterior['redshift'][:,:,np.argmin(separation)].median()
    spire_fluxes=np.mean(src_f[:,np.argmin(separation),:],axis=0)
    spire_fluxes_err=np.std(src_f[:,np.argmin(separation),:],axis=0)
    esb['f_spire_250'][i]=spire_fluxes[0]
    esb['f_spire_350'][i]=spire_fluxes[1]
    esb['f_spire_500'][i]=spire_fluxes[2]
    esb['ferr_spire_250'][i]=spire_fluxes_err[0]
    esb['ferr_spire_350'][i]=spire_fluxes_err[1]
    esb['ferr_spire_500'][i]=spire_fluxes_err[2]
    

No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit
No fit


In [19]:
esb_web=esb[1:10]

In [22]:
service = vo.dal.TAPService("https://herschel-vos.phys.sussex.ac.uk/__system__/tap/run/tap")
tables=[]
for src in range(0,len(esb_web)):
    resultset = service.search("SELECT TOP 10000 * FROM herschelhelp.main WHERE help_id = '"+str(esb_web['help_id'][src])+"'")
    tables.append(resultset.to_table())


In [23]:
ls_tables=vstack(tables)
cols_object=[c for c in ls_tables.columns if ls_tables[c].dtype =='O']
for c in cols_object:
    ls_tables[c]=ls_tables[c].astype(str)
#ls_tables.rename_column('help_id','id')

In [25]:
cols_keep=['help_id','ra','dec','redshift']+[c for c in ls_tables.colnames if 'megacam' in c]+[c for c in ls_tables.colnames if 'irac' in c]+[c for c in ls_tables.colnames if 'mips' in c]+[c for c in ls_tables.colnames if 'spire' in c]

In [26]:
for i in range(0,len(esb_web)):
    ls_tables['f_spire_250'][i]=1000.0*esb_web['f_spire_250'][i]
    ls_tables['f_spire_350'][i]=1000.0*esb_web['f_spire_350'][i]
    ls_tables['f_spire_500'][i]=1000.0*esb_web['f_spire_500'][i]
    ls_tables['ferr_spire_250'][i]=1000.0*esb_web['ferr_spire_250'][i]
    ls_tables['ferr_spire_350'][i]=1000.0*esb_web['ferr_spire_350'][i]
    ls_tables['ferr_spire_500'][i]=1000.0*esb_web['ferr_spire_500'][i]
    ls_tables['flag_spire_250'][i]=False
    ls_tables['flag_spire_350'][i]=False
    ls_tables['flag_spire_500'][i]=False
    ls_tables['redshift']=esb_web['redshift']

In [27]:
from herschelhelp.external import convert_table_for_cigale

In [28]:
new_table=convert_table_for_cigale(ls_tables[cols_keep])

In [36]:
data_file='/Volumes/pdh_storage/cigale/All_ESB.fits'
new_table.write(data_file,overwrite=True)

In [37]:
fin = open("/Volumes/pdh_storage/cigale/pcigale_obs_orig.ini")
fout = open("/Volumes/pdh_storage/cigale/pcigale.ini", "wt")
for line in fin:
    if 'data_file =' in line:
        fout.write('data_file = ' +data_file + ' \n')

    else:
        fout.write(line)
fin.close()
fout.close()
fin = open("/Volumes/pdh_storage/cigale/pcigale_obs_orig.ini.spec")
from shutil import copyfile
copyfile("/Volumes/pdh_storage/cigale/pcigale_obs_orig.ini.spec","/Volumes/pdh_storage/cigale/pcigale.ini.spec")
fin.close()

In [38]:
import subprocess
p = subprocess.Popen(['pcigale', 'run'], cwd='/Volumes/pdh_storage/cigale/')
p.wait()

0

In [39]:
from shutil import move, rmtree
rmtree('/Volumes/pdh_storage/cigale/fit_all_ESBs/')
move('/Volumes/pdh_storage/cigale/out/', '/Volumes/pdh_storage/cigale/fit_all_ESBs/')

'/Volumes/pdh_storage/cigale/fit_all_ESBs/'